In [1]:
!pip install gensim tqdm stop_words pymorphy2

In [2]:
!pip install python-telegram-bot==13.3

In [3]:
!pip install pandarallel

In [4]:
!pip install scikit-learn

In [5]:
!conda install -c conda-forge python-annoy -y

Solving environment: done

# All requested packages already installed.



In [6]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
import pickle
    
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
import mmap
import re

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [ ]:
# Скачать файл Otvety.txt по ссылке  (1,7G)
# https://drive.google.com/file/d/1DQL9ybca4USImUDaxxHmkIZNmClKBtKG/view

In [41]:
# Разделение файла вопросов-ответов на два файла, ограничим 500 тыс. строк
question = False
answer = False
counter = 0
i = 1
#counter_ans = 0
with open("question_test.txt", "w") as fout_q:
    with open("answers_test.txt", "w") as fout_ans:
        with open("Otvety.txt", "r") as fin:
            for line in fin:
                if counter > 500000:
                    break
                if line.startswith("---"):
                    if not question and not answer:
                        question = True
                        answer = False
                    else:
                        print(f'{i} warning {counter}')
                        question = False
                        answer = False
                        i += 1
                        continue
                    counter += 1
                    #print(counter)
                    continue
                if question and not answer:
                    question_line = line
                    #print(line)
                    question = False
                    answer = True
                    continue
                if answer and not question:
                    fout_ans.write(line)
                    fout_q.write(question_line)
                    answer = False
                    #print(line)
                    continue
print(counter)

1 warning 16008
2 warning 24252
3 warning 28063
4 warning 63206
5 warning 67461
6 warning 67998
7 warning 75922
8 warning 81640
9 warning 89370
10 warning 107202
11 warning 110664
12 warning 115360
13 warning 120971
14 warning 128226
15 warning 130296
16 warning 151831
17 warning 155515
18 warning 159384
19 warning 171829
20 warning 174883
21 warning 178879
22 warning 179555
23 warning 180490
24 warning 181882
25 warning 185220
26 warning 279098
27 warning 282378
28 warning 284824
29 warning 285086
30 warning 291145
31 warning 320844
32 warning 331618
33 warning 339317
34 warning 347842
35 warning 353087
36 warning 356344
37 warning 358027
38 warning 367174
39 warning 370476
40 warning 375469
41 warning 389756
42 warning 406089
43 warning 409547
44 warning 417512
45 warning 431590
46 warning 432454
47 warning 443862
48 warning 453486
49 warning 454997
50 warning 462469
51 warning 463497
52 warning 480454
53 warning 480939
54 warning 481799
55 warning 491150
56 warning 493268
57 warning

In [42]:
counter = 0
with open("answers_test.txt", "r") as fin:
    for line in fin:
        if line == '\n':
            counter += 1
        

print(counter)

0


In [43]:
counter = 0
with open("question_test.txt", "r") as fin:
    for line in fin:
        if line == '\n':
            counter += 1
        

print(counter)

0


In [44]:
print(get_num_lines("question_test.txt"))

499943


In [45]:
print(get_num_lines("answers_test.txt"))

499943


In [8]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [9]:
# Препроцессинг текста
def preprocess_txt(line):
    
    line = re.sub('<[^<]+?>', '', line) # удалить из входного текста все HTML-теги
    line = re.sub(r'http\S+', '', line) # удаляем из текста все URL и ссылки
    line = re.sub('[^а-яА-ЯёЁ0-9\s]', '', line) # убираем специальные символы: избавляемся от всего, что не является "словами"
    line = re.sub(r'[^\w\s]', '', line) # удаляем из текста все знаки препинания
    
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    
    return spls

In [48]:
# Обработка текста вопросов

sentences_q = []

file_path_from = 'question_test.txt'
file_path_to = 'question_test_sent.txt'

if not os.path.isfile(file_path_to):
    
    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'w') as fileto:
        with open(file_path_from) as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences_q.append(spls)
                fileto.write(' '.join(spls)+'\n')
    filefrom.close()
    fileto.close()

  0%|          | 0/499943 [00:00<?, ?it/s]

In [10]:
# Обработка текста ответов

sentences_ans = []

file_path_from = 'answers_test.txt'
file_path_to = 'answers_test_sent.txt'

if not os.path.isfile(file_path_to):
    
    N = get_num_lines(file_path_from)
    with open(file_path_to, mode = 'w') as fileto:
        with open(file_path_from) as filefrom:
            for k in tqdm(range(N)):
                line = filefrom.readline()
                if line == '': break
                spls = preprocess_txt(line)
                sentences_ans.append(spls)
                fileto.write(' '.join(spls)+'\n')
    filefrom.close()
    fileto.close()

  0%|          | 0/499943 [00:00<?, ?it/s]

In [11]:
# Загрузить результат из файла ответов

sentences_ans = []

file_path_from = 'answers_test_sent.txt'
if os.path.isfile(file_path_from):  
    N = get_num_lines(file_path_from) 
    with open(file_path_from, mode = 'r') as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences_ans.append(line.split())
    filefrom.close()

  0%|          | 0/499943 [00:00<?, ?it/s]

In [12]:
# Загрузить результат из файла вопросов

sentences_q = []

file_path_from = 'question_test_sent.txt'
if os.path.isfile(file_path_from):  
    N = get_num_lines(file_path_from) 
    with open(file_path_from, mode = 'r') as filefrom:
        for k in tqdm(range(N)):
            line = filefrom.readline()
            if line == '': break
            sentences_q.append(line.split())
    filefrom.close()

  0%|          | 0/499943 [00:00<?, ?it/s]

In [14]:
sentences_ans[:3]

[['хомячок'],
 ['вобщий', 'прикалывать', 'тема'],
 ['счастие',
  'свалиться',
  'хороший',
  'пойти',
  'милиция',
  'заявить',
  'находка',
  'деньга',
  'тероть',
  'самый',
  'интересный',
  'неприменный',
  'искать',
  'поверьте',
  'найти',
  'видеть',
  'подобный',
  'нарваться',
  'бабушка',
  'помочий',
  'внук',
  'покупка',
  'квартира',
  'бандит',
  'разговаривать',
  'иначе',
  'бабушка',
  'милиция',
  'выбор',
  'шанс',
  'подарок',
  'выше',
  'котрый',
  'никто',
  'спросить',
  'хороший',
  'отдать',
  'хотяб',
  '500',
  'благотворительность',
  'дабы',
  'спугнуть',
  'удача']]

In [16]:
sentences_q[:3]

[['вопрос', 'тдв', 'отдыхать', 'лично', 'советовать', 'завести'],
 ['парень',
  'относиться',
  'цветной',
  'линза',
  'девушка',
  'зелёный',
  'глаз',
  'голубой'],
 ['делать', 'найти', '2', 'миллион', 'рубль']]

In [17]:
# Обучим модель FastText

file_path_from = 'ft_model_test'
if not os.path.isfile(file_path_from):  
    
    modelFT = FastText(sentences=sentences_ans, vector_size=100, min_count=1, window=5)
    modelFT.save("ft_model_test")

In [18]:
# Загрузить модель

modelFT = FastText.load("ft_model_test")
ft_index = annoy.AnnoyIndex(100 ,'angular')

In [19]:
index_map = {i: ' '.join(sentences_ans[i]) for i in range(0, len(sentences_ans))}

In [20]:
index_map[3]

'эбу электронный блок управление двигатель автомобиль название контроллер принимать информация датчик обрабатывать особый алгоритм отталкиваться получить данные отдавать команда исполнительный устройство система'

In [21]:
with open("index_speaker.pkl", "wb") as f:
        pickle.dump(index_map, f)

In [22]:
# Создаем Индексы для вопросов-ответов

file_path_from = 'speaker.ann'
if not os.path.isfile(file_path_from):  
    modelFT = FastText.load("ft_model_test")
    ft_index = annoy.AnnoyIndex(100 ,'angular')

    counter = 0
      
    for sent in tqdm(sentences_q):
        n_ft = 0
        question = sent
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)

        counter += 1


    ft_index.build(10)
    ft_index.save('speaker.ann')
    
    #with open("index_speaker.pkl", "wb") as f:
        #pickle.dump(index_map, f)


  0%|          | 0/499943 [00:00<?, ?it/s]

In [23]:
#  Загрузим индексы
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')
index_map = pd.read_pickle("index_speaker.pkl")

In [24]:
index_map[3]

'эбу электронный блок управление двигатель автомобиль название контроллер принимать информация датчик обрабатывать особый алгоритм отталкиваться получить данные отдавать команда исполнительный устройство система'

In [25]:
np.random.permutation(100)

array([22, 66, 87, 21, 48, 29, 65,  7, 50, 79, 54, 61,  6, 51, 75,  9, 10,
       30, 26, 34, 39, 43, 41, 17,  4, 69, 73, 96, 99, 23, 85, 13, 93, 44,
        2, 24, 72, 68, 71, 63, 38, 97, 33, 47, 95, 57, 27, 32, 49, 81, 12,
       18,  5, 40, 82,  3, 58, 42, 88, 14, 98, 94, 70, 56, 84, 92, 59, 20,
       53,  1, 76, 28, 83, 37, 11, 45, 52, 64, 90, 60, 36, 15, 77, 67, 31,
       74, 16, 55,  0, 19, 62, 46, 89, 91,  8, 35, 80, 78, 86, 25])

In [28]:
#  Пример получения индексов
a = ft_index.get_nns_by_vector(np.random.permutation(100), 3, include_distances=True)
a

([429649, 356823, 110903],
 [1.097115397453308, 1.1190354824066162, 1.125589370727539])

In [29]:
[index_map[x] for x in a[0]]

['педофил надеяться',
 'джинсы водолазка',
 'ребёнок сын ребёнок любой пол идеали зароботок трое мальчик девочка']

In [ ]:
# https://gbcdn.mrgcdn.ru/uploads/asset/5209459/attachment/1b2f5aa57ff77e7c2d2ee26ceb09eb9e.csv

In [30]:
# Создадим модель продуктовых данных

shop_data = pd.read_csv("ProductsDataset.csv")
#shop_data = shop_data.iloc[:5000, :]

shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].progress_apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

  0%|          | 0/35548 [00:00<?, ?it/s]

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, новый, носить, реал, красивый,..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [31]:
# Подготовка для создания модели для определения домена данных

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))
#vectorizer = CountVectorizer()

In [32]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
# Вопрос-ответный домен
negative_texts = [" ".join(index_map[i]) for i in tqdm(idxs)]
# Продуктовый домен
positive_texts = [" ".join(val) for val in tqdm(shop_data['text'].values)]

  0%|          | 0/34288 [00:00<?, ?it/s]

  0%|          | 0/35548 [00:00<?, ?it/s]

In [33]:
# ВО = 0, Прод = 1

dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [35]:
vectorizer.fit(dataset)

CountVectorizer(ngram_range=(1, 2))

In [36]:
x_train_vec = vectorizer.transform(X_train)

In [39]:
x_test_vec = vectorizer.transform(X_test)

In [40]:
# Модель

#x_train_vec = vectorizer.transform(X_train)
#x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression(solver='lbfgs', max_iter=1000).fit(x_train_vec, y_train)

In [41]:
# Качество

from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.962557273768614

In [42]:
# Добавим IDF взвешивание (для каждого слова найдем IDF вес)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(dataset)

In [44]:
np.mean(tfidf_vect.idf_)

11.048852459258441

In [45]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}

In [46]:
len(idfs.keys())

41159

In [47]:
list(idfs.keys())[:5]

['юбка', 'детский', 'новый', 'носить', 'реал']

In [48]:
list(idfs.values())[:5]

[10.544481321830316,
 8.627558709648255,
 11.460772053704472,
 11.460772053704472,
 11.460772053704472]

In [49]:
modelFT = FastText.load("ft_model_test")

In [50]:
# Создаем Индексы для продуктовых данных

file_path_from = 'shop.ann'
if not os.path.isfile(file_path_from):  
    
    
    ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

    midf = np.mean(tfidf_vect.idf_)

    index_map_shop = {}
    counter = 0

    for i in tqdm(range(len(shop_data))):
        n_ft = 0
        index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
        vector_ft = np.zeros(100)
        for word in shop_data.loc[i, "text"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word] * idfs.get(word, midf)
                n_ft += idfs.get(word, midf)
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
            ft_index_shop.add_item(counter, vector_ft)
            counter += 1

    ft_index_shop.build(10)
    ft_index_shop.save('shop.ann')

    file_path_from = 'index_shop.pkl'
    if not os.path.isfile(file_path_from):  
    
        with open("index_shop.pkl", "wb") as f:
            pickle.dump(index_map_shop, f)


  0%|          | 0/35548 [00:00<?, ?it/s]

In [51]:
# Загрузим индексы

midf = np.mean(tfidf_vect.idf_)

ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann') 

index_map_shop = pd.read_pickle("index_shop.pkl")

In [83]:
# Основная функция преобразования текста в вектор х100

def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [ ]:
# def embed_txt(txt, idfs, midf):
#     n_ft = 0
#     vector_ft = np.zeros(100)
#     for word in txt:
#         if word in modelFT.wv:
#             vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
#             n_ft += 1 # idfs.get(word, midf)
#     return vector_ft / n_ft

In [57]:
# Создание своего бота в телеграмм
# @botfather
# /start
# /newbot - create a new bot
# name1
# name1_BOT
#. ->. API

In [91]:
# заменить на свой токен
updater = Updater("6228297259:AAHe1et_NeJbuN6-fnzzVV7w2l4L0CJPYpA", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')

def textMessage(update, context):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    # ПРОД
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    
    # QA
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    
    # 
    if distances[0] > 0.9:
        print(distances[0])
        context.bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        return
    
    # Вопрос-Ответ
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])
            
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

0.9157148003578186
